# **Lecture_2_2. Linear Regression from scratch using gradient descent**

Linear regression is one of the most popular tools for regression models.

Let us consider an example where we have to estimate the price of the house based on its area and age. To actually train a model, we would need a dataset of different house prices along with their area and age. 

In this dataset, area and age will be know as our `features` using which we will estimate the price. The price or the predicted value will be known as `label`.

We can use linear regression model for such a problem. Let's see what actually happens in linear regression and how we can implement it.

## **Linear Regression**




In this model, we have some independent variable values x (area and age in case of house price prediction) and a dependent variable y (price of the house) which is linearly dependent on x values. 

We try to generalize y as a weigthed sum of elemnts in x with some noise on the onservations.

$price = weight_{area} * area + weight_{age} * age + b$

Here b is the **bias**. The weights determine the influence of each feature on our prediction and the bias tells us what value the predicted price should take when all of the features take value 0. 

## **Loss Function**

We also need to compute loss. We use sum of squared error as our ***loss function***

## **Optimization**

We also need an **optimization algorithm** so that we can minimize our loss function. We will be using gradient descent.

**Let us summarize above points for linear regression:**

**Input**: $\boldsymbol{X}$ (features)

**Output**: $\mathbf{y}$ (label)

**Parameters**: $\mathbf{w}$, b (weights and bias)


The training examples are stored in matrix $\boldsymbol{X}$ row-wise <br><br>
$\boldsymbol{X} =  \begin{bmatrix}
x_1^{(1)}x_2^{(1)}\cdot\cdot\cdot x_n^{(1)} \\ x_1^{(2)}x_2^{(2)}\cdot\cdot\cdot x_n^{(2)}  \\ \cdot \\ \cdot \\ \cdot\ \\ x_1^{(m)}x_2^{(m)}\cdot\cdot\cdot x_n^{(m)} \end{bmatrix}$ , $ \mathbf{w} =\begin{bmatrix}
w_1  w_2  \cdot  \cdot  \cdot  w_n\end{bmatrix}$

We can calculale the predicted values of y for all m observations using follwoing:

\begin{equation}
\hat{\mathbf{y}}  = \boldsymbol{X} \mathbf{w^T} + b
\end{equation}
where $\boldsymbol{X} \mathbf{w^T}$ is matrix multiplication of $\boldsymbol{X}$ and $\mathbf{w^T}$ 

<b>Cost function</b>
\begin{equation}
\mathcal{L} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2 
\end{equation}

Cost Function (Vector Notation):
\begin{equation}
\mathcal{L}  = \frac{1}{m}(\mathbf{\hat{y}}  -\mathbf{y})^T (\mathbf{\hat{y}}  -\mathbf{y})
\end{equation}

Gradient of thr Loss Function with respect to $\mathbf{w}$:
\begin{equation}
\frac{2}{m}(\mathbf{\hat{y}}  -\mathbf{y})^T X
\end{equation}

Gradient of the Loss Function with respect to $b$:
\begin{equation}
\frac{2}{m}\mathbf{1_m}^T(\mathbf{\hat{y}}  -\mathbf{y}))
\end{equation}

here $\mathbf{1_m}$ denote the column vector of all ones and length m

Gradient Descent Algorithm:

The algorithm starts with some “initial guess” for $\mathbf{w}$ and $b$, and that repeatedly changes $\mathbf{w}$ and $b$ to make  $\mathcal{L}$  smaller, until we converge to a value of $\mathbf{w}$ and $b$ minimizes $\mathcal{L}$.

\begin{equation}
\mathbf{w^T} := \mathbf{w^T} - \frac{2\alpha} {m}\boldsymbol{X}^T(\mathbf{\hat{y}}  -\mathbf{y})
\end\{equation}


\begin{equation}
b := b - \frac{2\alpha} {m}\mathbf{1_m}^T(\mathbf{\hat{y}}  -\mathbf{y})
\end\{equation}




**We will implement linear regression algorithm from sratch in this notebook using the above formulas.**

In [ ]:
# Importing PyTorch Library
import torch

# Importing random library to generate random dataset
import random

In [ ]:
# To get deterministic results
torch.manual_seed(456)
random.seed(123)

We will be using W&B for visualization.

In [ ]:
# Install wandb and update it to the latest version
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 9.0 MB/s 
     |████████████████████████████████| 181 kB 69.1 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 157 kB 76.2 MB/s 
     |████████████████████████████████| 157 kB 77.4 MB/s 
     |████████████████████████████████| 157 kB 80.1 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 156 kB 77.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=90f940b906ee6333099b4cb6c023f7ec7b71b1b84ef03d31d02cdcbc2bc595e1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# Import wandb
import wandb

# Login to W&B
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Initialize a new project and run in wandb

In [ ]:
# we have specified the id for our run and set resume = None
# if we restart the run it will overwrite teh existing run
# If you want to resume runs then resume shoule be 'allow
wandb.init(name = "L_2_LR", project = 'dl22_l2')

wandb: Currently logged in as: hsingh-utd. Use `wandb login --relogin` to force relogin


## **Generating a Dataset**

We will generate a dummy dataset having 1000 observations and 2 features.
The observations are sampled from standard normal distribution.

Let us have our true parameter values to be w = [3, -4.5] and b = 5.2. 

`y = Xw.T + b + noise`

We will further assume that nose will be normally distributed with mean 0 and standard deviation of 0.01. 


In [ ]:
def generate_dataset(w, b, num): 

    """
    Function to generate a dataset. 
    Input parameter : 
    w: weights, 
    b: bias 
    num: number of obervations
    Output: feature and labels of a dataset
    """
    
    # Generate X values from standard normal distribution 
    X = torch.normal(0, 1, (num, len(w.T)))

    # Generate y values: y = Xw + b
    y = torch.mm(X, w.T) + b

    # Adding noise in labels
    
    y += torch.normal(0, 0.01, y.shape)

    # Returning the dataset generated
    return X, y.view((-1, 1))

In [ ]:
# Initializing actual weight and bias values
w_true = torch.Tensor([3, -4.5]).view(1,-1)
b_true = 5.2

# Calling the generate_dataset function to create a dummy dataset
features, labels = generate_dataset(w_true, b_true, 1000)

wandb.log({'plots/features': wandb.Histogram(features)})
wandb.log({'plots/labels': wandb.Histogram(labels)})

## **Visualization of the dataset**

In [ ]:
# Importing matplotlib library for visualizing dataset
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
fig.suptitle('Sactter Plots')

# We will create a scatter plot between feature and labels
ax1.scatter(features[:, 0].detach().numpy(), labels.detach().numpy(),s = 1)
ax1.set(xlabel='X1')
ax1.set(ylabel='y')
ax2.scatter(features[:, 1].detach().numpy(), labels.detach().numpy(), s = 1)
ax2.set(xlabel='X2')
ax2.set(ylabel='y')

wandb.log({"scatter plot ": plt})

From the above visualization, we can observe a linear relation between features and label values.

## **Reading the dataset**

For training our model, we will use mini batches from the dataset and use them to update our model.

To simplify this process, we will define a function which will shuffle the dataset and access it in mini batches.

In [ ]:
def read_data(batch_size, features, labels):
    """
    Function to access data in mini batches. 
    Input parameters: batch size, features, labels. 
    Output: iterator for minibatch data
    """
    
    # Number of examples in the datasets
    num = len(features)

    # Creating a list of indices from 0 to number of examples in the dataset
    indices = list(range(num))

    # The examples are read at random, in no particular order
    random.shuffle(indices)

    # Return the features and labels of the minibatch
    for i in range(0, num, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num)])
        yield features[batch_indices], labels[batch_indices]

Lets create a batch size of 15 and visualize the feature and data values better understanding.

In [ ]:
# Specify value for batch_size
batch_size = 15

# Call the function read_data and visualize the size of minibatch
for X, y in read_data(batch_size, features, labels):
    print(X.shape)
    print(y.shape)
    break

torch.Size([15, 2])
torch.Size([15, 1])


## **Linear Regression Model**

In [ ]:
def linear_reg(X, w, b):
  """
  Function for linear regression y = Xw + b. 
  Input parameter: features, weights, bias. 
  Output: predicted labels
  """
  return torch.mm(X,w.T) + b

## **Loss Function**

In [ ]:
def mse_loss(ypred, y):
  """
  Squared error loss function.
  Input: actual labels and predicted labels
  Output: squared error loss
  """
  error = ypred - y.view(ypred.shape)
  mean_squared_error = error.T.mm(error)/len(y)
  return mean_squared_error

## **Optimization Algorithm**

In [ ]:
# weight update step
def sgd(params, params_grad, lr):
    for param, param_grad in zip(params, params_grad):
        param -= lr * param_grad
        

## **Model Training**

**Model Training** involves five steps: 

- Step 0: Randomly initialize parameters / weights
- Step 1: Compute model's predictions - forward pass
- Step 2: Compute loss
- Step 3: Compute the gradients
- Step 4: Update the parameters
- Step 5: Repeat steps 1 - 4

Model training is repeating this process over and over, for many **epochs**.

An **epoch** is complete when we have used every point once for for computing the loss

We will use min-batch gardient descent. In minibatch gradient descent , we take a small batch of the data. We compute the loss for the batch and update the weights. **If there are $n$ batches, then in one epoch weights get updated $n$ times**. 

***Learning rate*** and ***epochs*** are known as hyperparameters. We have to fine tune the value of hy[erparametes using ceoss-validation.


In [ ]:
# Step 0: 
# Create a tensor weights with values from normal distribution
# We are calculating gardients manually
# Thus, we will set requires_grad to False

w = torch.normal(0, 0.01, size=(1, 2), requires_grad=False)

# Create a tensor for bias
b = torch.zeros(1, requires_grad=False)
print(w)

tensor([[-0.0104, -0.0055]])


In [ ]:
# Set the values for learning rate and number of epochs
lr = 0.005
epochs = 10

# Iterate over the whole dataset
for epoch in range(epochs):
    
    # Iterate over mini batch
    for X, y in read_data(batch_size, features, labels):

        # step 1 :forward pass - compute predictions
        ypred = linear_reg(X, w, b)

        # step 2: Calculate minibatch loss
        batch_loss = mse_loss(ypred, y)
        

        # step 3: Compute gradient on loss with respect to weights and bias
        grad_w = 2 *X.T.mm(ypred-y)/len(y)
        grad_b = 2 *(ypred-y).sum()/len(y)

        # step 4: Update parameters using their gradient using optimization algorithm
        sgd([w.T, b], [grad_w, grad_b], lr)  

    
    # Calculate and print loss for the complete epoch
    train_l = mse_loss(linear_reg(features, w, b), labels)

    # We can observe the epoch vs loss curve in W&B
    wandb.log({"/Loss_2_2": train_l.item()})
        
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 15.022364
epoch 2, loss 4.405335
epoch 3, loss 1.294533
epoch 4, loss 0.377739
epoch 5, loss 0.111390
epoch 6, loss 0.032954
epoch 7, loss 0.009864
epoch 8, loss 0.003008
epoch 9, loss 0.000970
epoch 10, loss 0.000364


We can observe that with each epoch, our loss is getting reduced hence our linear regression model is able to classify accurately.

Now since we generated the dataset ourselves we know the actual values for weights and bias, so we can check the error in both of them.

In [ ]:
# Printing error in weights and bias
print(f'Error in estimating w: {w_true.view(-1,1) - w}')
print(f'Error in estimating b: {b_true - b}')
print(f'estimated value of w: {w}')
print(f'estimated value of b: {b}')

Error in estimating w: tensor([[ 0.0080,  7.4879],
        [-7.4920, -0.0121]])
Error in estimating b: tensor([0.0088])
estimated value of w: tensor([[ 2.9920, -4.4879]])
estimated value of b: tensor([5.1912])


In [ ]:
wandb.finish()

/Loss_2_2,█▃▂▁▁▁▁▁▁▁
/Loss_2_2,0.00036


We can see that the error is minimal thus, our model is working well.